## Quick Start

This notebook could counted to testing the full cycle of training-inference loop of Llama 2-7b model on a single GPU.With int8 quantization and LoRA the minimal requrements are 24GB, but I reccomend to use A100 with at least 40GB to perform all kinds of testing and disable quantization for higher accuracy

### Step 0: Install pre-requirements

In [1]:
!git clone https://github.com/Coldra1n/Thesis_LLaMA2.git

Cloning into 'Thesis_LLaMA2'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (43/43), 25.57 KiB | 12.79 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
%cd /content/Thesis_LLaMA2

/content/Thesis_LLaMA2


In [3]:
!pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-0i6g6f5h
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-0i6g6f5h
  Resolved https://github.com/huggingface/peft.git to commit 0c9354bda98eb7f5348699e23ab752e8dca1e60e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
#uncomment in case of errors with install requirements
# %%bash
#!pip install transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets

### Step 1: Load the model



In [4]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="NousResearch/Llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)#or torch.bfloat16


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Step 2: Load the preprocessed dataset


In [10]:
from pathlib import Path
import os
import sys
from Utils.dataset_utils import get_preprocessed_dataset
from Configs.datasets import get_scientific_papers_dataset

train_dataset = get_preprocessed_dataset(tokenizer, scientific_papers_dataset, 'train')

ImportError: ignored

### Step 3: Check base model

Run the base model on an example input:

In [ ]:
article = """Summarize this article:
world - wide , infertility affects 1015% of couples who are trying to conceive , and about 15% of these cases are caused by male factors , which affect 1 out of 20 men in the general population . most cases of male infertility are idiopathic , apart from several etiologies , such as obstruction of deferent duct , varicocele , sexual dysfunction , and cryptorchidism . although assisted reproductive technology ( art ) has helped many sterile couples to conceive , non - obstructive azoospermia ( noa ) , which accounts for a considerable proportion of male infertility , has a dramatically lower rate of sperm retrieval and clinical pregnancy . the etiological mechanism of noa is unknown , but factors such as oxidative stress were considered to have effects on spermatogenesis , and some antioxidants have been effective in protecting spermatogenesis . therefore , it is helpful to explore the underlying pathogenesis of noa in these patients . micrornas are a class of small rnas that do not code amino acid sequences , but they play fundamental roles in regulating gene expression after transcription . lian et al . found 154 down - regulated mirnas and 19 up - regulated mirnas in testes of noa patients compared to fertile males , by using microarray technologies . furthermore , some of the mirnas have been shown to affect the proliferation , apoptosis , and dna damage in germ cells [ 911 ] . mir-210 is one of the 19 up - regulated mirnas in testes of noa patients , located within the genomic loci of transcript ak123483 . it can be induced by hypoxia , and plays an essential role in cell adaptation to hypoxia . mir-210 also affects regulation of diverse physiological processes , such as angiogenesis , cell survival , proliferation , cell cycle arrest , protein modification , and dna damage repair . although mir-210 has been shown to be involved in regulation of physiological processes in various diseases and to be an up - regulated mirna in testes of noa patients , it remains unknown how mir-210 affects spermatogenesis . hence , the aim of this study was to investigate the underlying mechanisms by which mir-210 is involved in the pathogenesis of spermatogenesis . we enrolled 25 patients ( aged 1841 years ) with azoospermia ( proven by 3 semen analyses from testicular biopsies from the first affiliated hospital of anhui medical university ) . pathological examinations were performed on each testicular specimen . combined with clinical features , 4 patients were diagnosed as having sertoli - cell - only syndrome ( scos ) , 7 patients were diagnosed as having maturation arrest ( ma ) , 8 patients were diagnosed as having hypospermatogenesis , and the other 6 patients were diagnosed as having obstructive azoospermia ( oa ) . all patients provided informed consent before their participation in this study . our local medical ethics committee approved this study before it began . to examine the location of insulin - like growth factor ii ( igf2 ) in human testicular tissues , we performed immunohistochemistry staining to detect the igf2 expression . tissues were cut into sections for immunoperoxidase staining after being treated with 4% pfa and paraffin wax . after the specific treatment with standard - procedure immunohistochemistry staining as described as lian et al . , sections were incubated with igf2 antibody ( abcam ) overnight at 4c and biotinylated secondary antibody ( abcam ) for 2 h at room temperature . to detect expression of mir-210 , rnas were extracted from nt-2 cells or tissues and subjected to real - time pcr as described as lian et al . . briefly , rna extraction was performed following a standard trizol protocol , real - time pcr was carried out with the abi step one system ( applied biosystems ) , and the sybr premix ex taq ii kit ( takara bio , inc . ) was used . primers for q - rt pcr were as follows : forward primer : 5-caataactgtgcgtgtgacagc-3 reverse primer : 5-tatggttttgacgactgtgtgat-3 forward primer : 5-cagcacatatactaaaattggaacg-3 reverse primer : 5-acgaatttgcgtgtcatcc-3 western blot analysis was carried out to detect protein expression of igf2 in the human testicular tissues in the 3 groups and in nt2 cells . anti - igf2 ( abcam ) was used for western blot analysis , and we used -actin as a loading control to detect expression of igf2 . we supplemented the medium with 10% fetal bovine serum ( life technology inc . ) , 1% antibiotics ( 100 units / ml penicillin , and 100 ug / ml streptomycin , life technology inc . ) . cells were incubated at 37c in a humidified incubator with 5% co2 . to transfect oligonucleotides and plasmids into nt-2 cells , lipofectamine rnaimax ( invitrogen ) and fugene hd ( roche ) all processes were performed in accordance with the protocols supplied by manufacturers . in this study , all the experiments were performed independently at least 3 times . we enrolled 25 patients ( aged 1841 years ) with azoospermia ( proven by 3 semen analyses from testicular biopsies from the first affiliated hospital of anhui medical university ) . pathological examinations were performed on each testicular specimen . combined with clinical features , 4 patients were diagnosed as having sertoli - cell - only syndrome ( scos ) , 7 patients were diagnosed as having maturation arrest ( ma ) , 8 patients were diagnosed as having hypospermatogenesis , and the other 6 patients were diagnosed as having obstructive azoospermia ( oa ) . all patients provided informed consent before their participation in this study . to examine the location of insulin - like growth factor ii ( igf2 ) in human testicular tissues , we performed immunohistochemistry staining to detect the igf2 expression . tissues were cut into sections for immunoperoxidase staining after being treated with 4% pfa and paraffin wax . after the specific treatment with standard - procedure immunohistochemistry staining as described as lian et al . , sections were incubated with igf2 antibody ( abcam ) overnight at 4c and biotinylated secondary antibody ( abcam ) for 2 h at room temperature . rnas were extracted from nt-2 cells or tissues and subjected to real - time pcr as described as lian et al . . briefly , rna extraction was performed following a standard trizol protocol , real - time pcr was carried out with the abi step one system ( applied biosystems ) , and the sybr premix ex taq ii kit ( takara bio , inc . ) was used . primers for q - rt pcr were as follows : forward primer : 5-caataactgtgcgtgtgacagc-3 reverse primer : 5-tatggttttgacgactgtgtgat-3 forward primer : 5-cagcacatatactaaaattggaacg-3 reverse primer : 5-acgaatttgcgtgtcatcc-3 western blot analysis was carried out to detect protein expression of igf2 in the human testicular tissues in the 3 groups and in nt2 cells . anti - igf2 ( abcam ) was used for western blot analysis , and we used -actin as a loading control to detect expression of igf2 . we supplemented the medium with 10% fetal bovine serum ( life technology inc . ) , 1% antibiotics ( 100 units / ml penicillin , and 100 ug / ml streptomycin , life technology inc . ) . cells were incubated at 37c in a humidified incubator with 5% co2 . to transfect oligonucleotides and plasmids into nt-2 cells , lipofectamine rnaimax ( invitrogen ) and fugene hd ( roche ) the igf2 gene is part of a cluster of imprinted genes expressing the single polypeptide as igf2 , which is only produced from the paternal allele . the maternal allele is transcriptionally silent . to clarify the location of igf2 in human testicular tissues , we found igf2 located in spermatocytes in the testes of patients with oa ( figure 1 ) . because igf2 is located in spermatocytes of the testis , we detected the expression of igf2 in cases with ma , hypospermatogenesis , and oa , but not in the scos patients . we found that igf2 was down - regulated in patients with ma and hypospermatogenesis compared to oa patients , which was considered as the control group with normal spermatogenesis , although without a significant difference ( figures 2 , 3 ) , possibly because there were fewer samples and longer preservation times of some samples . quantitative real - time pcr was performed to examined mir-210 expression in the testis of patients with ma , hypospermatogenesis , and oa . we found that mir-210 was significantly up - regulated in the testis of ma and hypospermatogenesis patients compared to oa patients ( figure 4 ) . however , due to errors in the rna extraction in the preliminary experiment , 3 testis samples ( 1 each ) from ma , hypospermatogenesis , and oa patients were damaged and were not tested . in the targetscan database , because the 3utr of the igf2-mrna has a putative mir-210-binding site , igf2 was predicted to be a potential target of mir-210 . to identify whether the igf2 gene was targeted by mir-210 directly , renilla luciferase reporters , which include the wild - type full - length 3utr forms of mir-210 seeding sites , figure 5 shows that there was a 60% decrease in luciferase activity after cotransfection of the mir-210 mimic and the renilla luciferase reporters into nt2 cells , and inhibiting mir-210 expression increased activity of the reporter renilla luciferase . expression of igf2 protein was also significantly lower in the nt2 cells transfected with mir-210 mimics than in control cells , and knockdown of mir-210 with mir-210 inhibitor increased protein expression of igf2 ( figures 6 ) . the igf2 gene is part of a cluster of imprinted genes expressing the single polypeptide as igf2 , which is only produced from the paternal allele . the maternal allele is transcriptionally silent . to clarify the location of igf2 in human testicular tissues , we found igf2 located in spermatocytes in the testes of patients with oa ( figure 1 ) . because igf2 is located in spermatocytes of the testis , we detected the expression of igf2 in cases with ma , hypospermatogenesis , and oa , but not in the scos patients . we found that igf2 was down - regulated.in patients with ma and hypospermatogenesis compared to oa patients , which was considered as the control group with normal spermatogenesis , although without a significant difference ( figures 2 , 3 ) , possibly because there were fewer samples and longer preservation times of some samples . quantitative real - time pcr was performed to examined mir-210 expression in the testis of patients with ma , hypospermatogenesis , and oa . we found that mir-210 was significantly up - regulated in the testis of ma and hypospermatogenesis patients compared to oa patients ( figure 4 ) . however , due to errors in the rna extraction in the preliminary experiment , 3 testis samples ( 1 each ) from ma , hypospermatogenesis , and oa patients were damaged and were not tested . in the targetscan database , because the 3utr of the igf2-mrna has a putative mir-210-binding site , igf2 was predicted to be a potential target of mir-210 . to identify whether the igf2 gene was targeted by mir-210 directly , renilla luciferase reporters , which include the wild - type full - length 3utr forms of mir-210 seeding sites , were used . figure 5 shows that there was a 60% decrease in luciferase activity after cotransfection of the mir-210 mimic and the renilla luciferase reporters into nt2 cells , and inhibiting mir-210 expression increased activity of the reporter renilla luciferase . expression of igf2 protein was also significantly lower in the nt2 cells transfected with mir-210 mimics than in control cells , and knockdown of mir-210 with mir-210 inhibitor increased protein expression of igf2 ( figures 6 ) . during recent decades several studies have focused on the effects of mirnas on spermatogenesis in male infertility [ 911,17 ] . however , it was not understood how mir-210 , which is one of the up - regulated mirnas in testes of patients with noa , was involved in spermatogenesis in male infertility . the transformation of diploid spermatogonia into mature haploid cells in spermatogenesis is a complex biological process in the testes of males . the insulin / igf system takes part in the processes of cell proliferation , cell growth , differentiation , and survival , which affects nearly every organ in the body . also , insulin / igf plays an important role in the proper function of the testis in males . igf2 binds to igf1r and insr - a with a high affinity and binds to insr - a / igf1r , insr - b / igf1r , but with lower affinity . found that in inactivated insr and igf1r , there was a 79% reduction in daily sperm production in adult mouse testes by a conditional ko approach . taken together , the aforementioned data suggest that igf2 might be involved in the process of spermatogenesis . to examine the specific mechanism by which mir-210 is associated with the process of spermatogenesis , quantitative real - time pcr was performed to detect mir-210 expression . we found that mir-210 was significantly up - regulated in the testes of subjects with ma and hypospermatogenesis patients compared to oa . these results agree with findings of lian et al . using microarray technologies performed in noa and normal controls . several studies have suggested that this mirna could be mediated by hypoxia and participate in various types of regulation of angiogenesis , cell survival , proliferation , cell cycle arrest , and protein modification [ 1214 ] . furthermore , some researchers even found that mir-210 might be considered as one of the indicated markers in some diseases , such as clear cell renal cell carcinoma and acute myeloid leukemia . in the present study we found that igf2 was targeted by mir-210 directly in the in vitro experiment in nt2 cells , and mir-210 might be associated with spermatogenesis by targeting igf2 in male infertility . firstly , as some errors occurred in the rna extraction in the preliminary experiment , mir-210 of 3 testes samples were damaged and not detected in the subsequent quantitative real - time pcr experiment , which might have affected our results . secondly , we did not investigate the functions of mir-210 and igf2 in vitro or in vivo , and we plan to do this in future research . we demonstrated that mir-210 might be associated with spermatogenesis by targeting igf2 in male infertility . future mechanistic studies on the role of mir-210/igf2 in the process of spermatogenesis in male infertility will provide new insights into the diagnosis and management of male infertility .
---
Summary:
"""

In [ ]:
model_input = tokenizer(article, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True))

We can see that the base model only repeats the conversation.

### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [ ]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=64, #8,32,64 the n_train. params
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)



### Step 5: Define an optional profiler

In [ ]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)

    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler

        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Step 6: Fine tune the model

Here, we fine tune the model for a single epoch which takes a bit more than an hour on a A100.

In [ ]:
from transformers import default_data_collator, Trainer, TrainingArguments



# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()

**TEST results**
Max CUDA memory allocated was 29 GB
Max CUDA memory reserved was 33 GB
Peak active CUDA memory was 29 GB
Cuda Malloc retires : 0
CPU Total Peak Memory consumed during the train (max): 7 GB
evaluating Epoch: 100% 828/828 [24:10<00:00,  1.75s/it]
 eval_ppl=tensor(4.3908, device='cuda:0') eval_epoch_loss=tensor(1.4795, device='cuda:0')
we are about to save the PEFT modules
PEFT modules are saved in /content/llama_PubMed directory
best eval loss on epoch 0 is 1.4795011281967163
Epoch 1: train_perplexity=4.3647, train_epoch_loss=1.4735, epcoh time 11230.147269584999s
Key: avg_train_prep, Value: 4.364653587341309
Key: avg_train_loss, Value: 1.4735387563705444
Key: avg_eval_prep, Value: 4.390754699707031
Key: avg_eval_loss, Value: 1.4795011281967163
Key: avg_epoch_time, Value: 11230.147269584999
Key: avg_checkpoint_time, Value: 0.0409323999992921
CPU Total Peak Memory consumed during the train (max): 7 GB
evaluating Epoch: 100% 828/828 [24:10<00:00,  1.75s/it]
 eval_ppl=tensor(4.3908, device='cuda:0') eval_epoch_loss=tensor(1.4795, device='cuda:0')
we are about to save the PEFT modules
PEFT modules are saved in /content/llama_PubMed directory
best eval loss on epoch 0 is 1.4795011281967163
Epoch 1: train_perplexity=4.3647, train_epoch_loss=1.4735, epcoh time 11230.147269584999s
Key: avg_train_prep, Value: 4.364653587341309
Key: avg_train_loss, Value: 1.4735387563705444
Key: avg_eval_prep, Value: 4.390754699707031
Key: avg_eval_loss, Value: 1.4795011281967163
Key: avg_epoch_time, Value: 11230.147269584999
Key: avg_checkpoint_time, Value: 0.0409323999992921


### Step 7:
Save model checkpoint

In [ ]:
model.save_pretrained(output_dir)

The final model - coldra1n/LLaMA2_PubMed_Final


### Step 8:
Try the fine tuned model on the same example again to see the learning progress:

In [ ]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=516)[0], skip_special_tokens=True))
